In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Gerekli kütüphanelerin yüklenmesi ...
!pip install transformers
!pip install scikit-learn
!pip install wandb
!pip install simpletransformers

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.2)


In [ ]:
# GPU kontrolü ...
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# Veri kümesinin yüklenmesi ve etiketlerin basılması ...
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/verikumesi.csv', delimiter=';', header=None, names=['sentence', 'word', 'tag'])
df = df.fillna(method="ffill")

df.head(10)
df['tag'].unique()

array(['TAG', 'O', 'B-LAW', 'I-LAW', 'B-PERSON', 'B-RIGHT',
       'B-ORGANIZATION', 'I-ORGANIZATION', 'B-PUNISHMENT', 'I-PERSON',
       '#', 'B-P_TERM', 'B-TIME', 'I-TIME', 'B-OFFENSIVE_WEAPON',
       'I-OFFENSIVE_WEAPON', 'I-PUNISHMENT', 'B-GPE', 'B-LOC', 'I-LOC',
       'B-NORP', 'B-ELEMENT_OF_CRIME', 'I-GPE', 'I-ELEMENT_OF_CRIME',
       'I-P_TERM', 'B-EQUIPMENT', 'B-MONEY', 'I-MONEY', 'I-RIGHT',
       'B-DATE', 'I-EQUIPMENT'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForTokenClassification

df["sentence"] = LabelEncoder().fit_transform(df["sentence"])

df.head(10)

,sentence,word,tag
0,0,WORD,TAG
1,1,Temel,O
2,1,İlkeler,O
3,1,ve,O
4,1,Tanımlar,O
5,1,Ceza,B-LAW
6,1,Kanununun,I-LAW
7,1,amacı,O
8,1,",",O
9,1,kişi,B-PERSON


In [ ]:
# Test ve train ayrımının yapılması ...
df.rename(columns={"sentence": "sentence_id", "word":"words", "tag":"labels"}, inplace=True)
df["labels"] = df["labels"].str.upper()

X = df[["sentence_id","words"]]
Y = df["labels"]

x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2)

train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

train_data

,sentence_id,words,labels
34808,322,.,O
1268,1947,veya,O
52427,1326,Akıl,O
10961,1927,ile,O
18376,2165,görülmesi,O
...,...,...,...
44820,893,her,O
46719,1006,.,O
12739,1984,halinde,O
23976,2375,davranışları,O


In [ ]:
from simpletransformers.ner import NERModel, NERArgs

label = df["labels"].unique().tolist()
print(label)

## Model için gerekli olan parametrelerin oluşturulması ...
## Bu değerler yapılan testlere göre belirlenmiştir. Bu testlerin sonuçları bitirme sunumlarında karşılaştırmalı olarak belirtilmiştir.
args = NERArgs()
args.num_train_epochs = 3
args.learning_rate = 3e-5
args.overwrite_output_dir = True
args.train_batch_size = 8
args.eval_batch_size = 32
args.output_dir = "/content/drive/MyDrive/Bitirme/Model5"
args.save_model_every_epoch = False;


## Gerekli modelin indirilmesi, model yolu daha sonra değiştirilebilir.
model = NERModel('bert','/content/drive/MyDrive/Bitirme/Model5', labels = label, args = args)
#model = NERModel('bert','bert-base-128k-cased', labels = label, args = args)




['TAG', 'O', 'B-LAW', 'I-LAW', 'B-PERSON', 'B-RIGHT', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-PUNISHMENT', 'I-PERSON', '#', 'B-P_TERM', 'B-TIME', 'I-TIME', 'B-OFFENSIVE_WEAPON', 'I-OFFENSIVE_WEAPON', 'I-PUNISHMENT', 'B-GPE', 'B-LOC', 'I-LOC', 'B-NORP', 'B-ELEMENT_OF_CRIME', 'I-GPE', 'I-ELEMENT_OF_CRIME', 'I-P_TERM', 'B-EQUIPMENT', 'B-MONEY', 'I-MONEY', 'I-RIGHT', 'B-DATE', 'I-EQUIPMENT']


In [ ]:
## Model eğitiminin başlatılması ...
model.train_model(train_data, eval_data=test_data, acc= accuracy_score)

(960, 0.039816434368610015)

In [ ]:
## Test değerlerin modele sokulması ...
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: # seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [ ]:
## Model sonuçlarının gösterilmesi ...
result

{'eval_loss': 0.19319900183472782,
 'f1_score': 0.8934993084370677,
 'precision': 0.9065757818765036,
 'recall': 0.8807947019867549}

In [ ]:
#from simpletransformers.ner import NERModel, NERArgs
#model = NERModel('bert','/content/drive/MyDrive/Bitirme/Model5')

In [1]:
## İstenilen cümlede tahmin yapılmasını sağlamak ...
prediction, model_output = model.predict(["Fail yararına cezayı hafifletecek takdiri nedenlerin varlığı halinde, ağırlaştırılmış müebbet hapis cezası yerine, müebbet hapis; müebbet hapis cezası yerine, yirmibeş yıl hapis cezası verilir. Diğer cezaların altıda birine kadarı indirilir."])

prediction

NameError: ignored